# Importation

In [18]:
import requests, json, pickle, time
import os
from telegram.ext import *
import telegram
import pandas as pd
import time
import json, pickle

# Variable

In [10]:
'https://www.binance.com/gateway-api/v1/friendly/pos/union?pageSize=50&pageIndex=1&status=ALL'

'https://www.binance.com/bapi/earn/v1/friendly/lending/project/list?pageSize=100&pageIndex=1&status=ALL'
defi_URL = 'https://www.binance.com/bapi/earn/v1/friendly/defi-pos/union?pageSize=50&pageIndex=1&status=ALL'
locked_URL = 'https://www.binance.com/gateway-api/v1/friendly/pos/union?status=ALL&pageSize=100'
with open('env.json', 'r') as fp:
    env = json.load(fp)

URL = os.getenv("URL")
bot_token = os.getenv("bot_token")
bot_chatID = os.getenv("bot_chatID")

In [11]:
API = env["BinanceStakeController"]

In [12]:
bot = telegram.Bot(token=API)
bot.sendMessage(chat_id = 1730011637, text = 'Hey there!')

# locked
response = json.loads(requests.get(locked_URL).text)["data"]
result = {}
for item in response:
    duration = []
    for asset in item["projects"]:
        duration.append(asset["duration"])
    result[item["asset"]] = duration
json.dump(result, open("data/locked.json", "w"))


# defi
response = json.loads(requests.get(defi_URL).text)["data"]
result = []
for item in response:
    result.append(item["asset"])
json.dump(result, open("data/defi.json", "w"))

# fonction

In [49]:
def add_user(update):
    chat_id = update.message.chat_id
    df = pd.read_csv("data/users.csv")
    if chat_id not in df.id.values:
        username = update.effective_user.username
        firstname = update.effective_user.first_name
        lastname = update.effective_user.last_name
        with open("data/users.csv", "a+") as f:
            f.write(f"\n{time.asctime()},{chat_id},{username},{firstname},{lastname}")
            f.close()
        update.message.reply_text("Welcome new user !")

def subscribe_error(update):
    update.message.reply_text(
        "Please enter at least one currency to subscribe ! \nIf you need some /help. \nOr contact @tttienthinh"
    )

def get_user(chat_id):
    df = pd.read_csv("data/locked.csv")
    locked = df[df.id == chat_id]
    
    df = pd.read_csv("data/defi.csv")
    defi = df[df.id == chat_id]
    return locked, defi

In [80]:
def responses(input_text):
    return input_text

def start_command(update, context):
    add_user(update)
    help_command(update, context)

def help_command(update, context):
    update.message.reply_text(open("text/help.txt", "r").read())

def settings_command(update, context):
    chat_id = update.message.chat_id
    
    response = open("text/settings1.txt", "r").read()
    locked, defi = get_user(chat_id)
    for token in list(locked.token.drop_duplicates()):
        response += token \
            + " - {" \
            + ",".join([str(x) for x in list(locked[locked.token == token].duration)]) \
            + "}\n"
    response += "\n"
    response += open("text/settings2.txt", "r").read()
    response += "\n".join([str(x) for x in list(defi.token)])
    response += "\n"
    update.message.reply_text(response)
    
def list_command(update, context):
    response = "Currencies for locked staking:\n\n"
    dico = json.load(open("data/locked.json", "r"))
    for token in sorted(list(dico.keys())):
        response += token + " - {" + ",".join(dico[token]) + "}\n"
    
    response += "\nCurrencies for DeFi staking:\n\n"
    dico = json.load(open("data/defi.json", "r"))
    response += "\n".join(sorted(dico))
    update.message.reply_text(response)
    
def subscribe_command(update, context):
    liste = context.args
    if len(liste) == 0:
        subscribe_error(update)
    else:
        dico = json.load(open("data/locked.json", "r"))
        chat_id = update.message.chat_id
        locked, defi = get_user(chat_id)
        for item in liste:
            item = item.replace(" ", "").upper()
            durations = []
            if "-" in item:
                i = item.index("-")
                token = item[:i]
                if token not in dico.keys():
                    update.message.reply_text(
                        f"Token {token} not found"
                    )
                else:
                    for duration in item[i+1:].split(","):
                        if (duration in dico[token]) and (duration not in durations):
                            durations.append(duration)
                        else:
                            update.message.reply_text(f"Token {token} not found")
            else:
                token = item
                if token not in dico.keys():
                        update.message.reply_text(f"Token {token} not found")
                else:
                    durations = dico[token]
            with open("data/locked.csv", "a+") as f:
                for duration in durations:
                    if ((locked.token==token) & (locked.duration==int(duration))).sum() == 0:
                        f.write(f"\n{chat_id},{token},{duration}")
                        update.message.reply_text(
                            f"Token {token}-{duration} subscribed successfully"
                        )
                    else:
                        update.message.reply_text(
                            f"Token {token}-{duration} already subscribed"
                        )
                f.close()
    
def subscribe_defi_command(update, context):
    liste = context.args
    if len(liste) == 0:
        subscribe_error(update)
    else:
        dico = json.load(open("data/defi.json", "r"))
        chat_id = update.message.chat_id
        locked, defi = get_user(chat_id)
        for token in liste:
            token = token.replace(" ", "").upper()
            if token not in dico:
                update.message.reply_text(
                    f"Token {token} not found"
                )
            else:
                with open("data/defi.csv", "a+") as f:
                    if (defi.token==token).sum() == 0:
                        f.write(f"\n{chat_id},{token}")
                        update.message.reply_text(
                            f"Token {token} subscribed successfully"
                        )
                    else:
                        update.message.reply_text(
                            f"Token {token} already subscribed"
                        )
                    f.close()
                
def unsubscribe_command(update, context):
    update.message.reply_text(
        "I'm sorry, this function is not created yet, I am working on it"
    )        
def unsubscribe_defi_command(update, context):
    liste = context.args
    if len(liste) == 0:
        subscribe_error(update)
    else:
        chat_id = update.message.chat_id
        df = pd.read_csv("data/defi.csv")
        for token in liste:
            token = token.replace(" ", "").upper()
            df.drop(
                labels=df[(df.token==token) & (df.id==chat_id)].index,
                axis=0,
                inplace=True
            )
        df.to_csv("data/defi.csv", index=False)
        update.message.reply_text(
            "Good, see /settings "
        )
    
def test(update, context):
    pass
    
def error(update, context):
    update.message.reply_text(
        "I'm sorry there was an error \nIf you need some /help. \nOr contact @tttienthinh"
    )
    print("Error")

# run

In [ ]:
updater = Updater(API, use_context=True)
dp = updater.dispatcher

dp.add_handler(CommandHandler("start", start_command))
dp.add_handler(CommandHandler("help", help_command))
dp.add_handler(CommandHandler("settings", settings_command))
dp.add_handler(CommandHandler("list", list_command))
dp.add_handler(CommandHandler("subscribe", subscribe_command))
dp.add_handler(CommandHandler("subscribe_defi", subscribe_defi_command))
dp.add_handler(CommandHandler("unsubscribe", unsubscribe_command))
dp.add_handler(CommandHandler("unsubscribe_defi", unsubscribe_defi_command))
dp.add_handler(CommandHandler("test", test))


dp.add_error_handler(error)

updater.start_polling()
updater.idle()

# Always on

In [2]:
def data():
    # Locked
    response = json.loads(requests.get(locked_URL).text)["data"]
    result = {}
    locked_avbl = []
    for item in response:
        duration = []
        for asset in item["projects"]:
            duration.append(asset["duration"])
            if not asset["sellOut"]:
                locked_avbl.append({
                    "asset":asset["asset"],
                    "duration":asset["duration"],
                    "APY":str(round(
                        float(asset["config"]["annualInterestRate"]) * 100,
                        2
                    ))
                })
        result[item["asset"]] = duration
    json.dump(result, open("data/locked.json", "w"))
    
    # Defi
    response = json.loads(requests.get(defi_URL).text)["data"]
    result = []
    defi_avbl = []
    for item in response:
        result.append(item["asset"])
        if not asset["sellOut"]:
            defi_avbl.append({
                "asset":asset["asset"],
                "duration":asset["duration"],
                "APY":str(round(
                    float(asset["config"]["annualInterestRate"]) * 100,
                    2
                ))
            })
    json.dump(result, open("data/defi.json", "w"))
    
    return locked_avbl, defi_avbl

In [4]:
def always_on():
    while True:
        total_messages = [[], []]
        start = time.time()
        past_locked_avbl = pickle.load(open('data/locked_avbl.pkl', 'rb'))
        past_defi_avbl = pickle.load(open('data/defi_avbl.pkl', 'rb'))
        locked_avbl, defi_avbl = data()
        # Locked
        for data in locked_avbl:
            if data not in past_result:
                df = pd.read_csv("data/locked.csv")
                token = data["asset"]
                duration = data["duration"]
                message = " ".join([
                    token,
                    f'{duration} days',
                    f'{data["APY"]}% APY',
                ])
                total_messages[0].append(message)
                message = "Locked Staking new release \n" + message
                for chat_id in list(df[(df.token==token) & (df.duration==int(duration))].id):
                    bot.sendMessage(
                        chat_id = chat_id, 
                        text = message
                    )
        # Defi
        for data in defi_avbl:
            if data not in past_result:
                df = pd.read_csv("data/defi.csv")
                token = data["asset"]
                message = " ".join([
                    token,
                    f'{data["APY"]}% APY',
                ])
                total_messages[1].append(message)
                message = "Defi Staking new release \n" + message
                for chat_id in list(df[df.token==token].id):
                    bot.sendMessage(
                        chat_id = chat_id, 
                        text = message
                    )
        pickle.dump(locked_avbl, open('data/locked_avbl.pkl', 'rb'))
        pickle.dump(defi_avbl, open('data/defi_avbl.pkl', 'rb'))
        end = time.time()
        time.sleep(max(60, 10*60 + now - end))  # Executing every 10 minutes

NameError: name 'URL' is not defined

In [34]:
[[],[]] ==[[],[]] 

True